# Implementation of some features based on POS patterns and sentiment

In [15]:
import nltk as nl
import pandas as pd
import numpy as np
import twython
import re
from nltk.sentiment import SentimentAnalyzer # sentiment analysis
from nltk.tokenize import RegexpTokenizer #tokenization
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer # sentiment analysis VADER
from nltk.tokenize import sent_tokenize #sentence tokenizer
#nl.download('vader_lexicon')
#nl.download('averaged_perceptron_tagger') for POS
#nl.download('punkt')

In [17]:
TrainingSet = pd.read_json("train1703.jsonl", lines = True)

In [19]:
TrainingSet

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle
0,608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...
1,609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...
2,609504474621612032,[],[U.S. Soccer should start answering tough ques...,Fri Jun 12 23:36:05 +0000 2015,[US to vote for Ali in FIFA election and not B...,A U.S. Senator's scathing letter questioned U....,,"[WINNIPEG, Manitoba â€“ The bubble U.S. Soccer...",U.S. Soccer should start answering tough quest...
3,609748367049105408,[],[How theme parks like Disney World left the mi...,Sat Jun 13 15:45:13 +0000 2015,"[Some 1,000 persons turned out in Albuquerque,...","America's top family vacation spots, like the ...","disney, disney world, disney ticket prices, di...",[When Walt Disney World opened in an Orlando s...,How theme parks like Disney World left the mid...
4,608688782821453824,[media/608688782821453825.jpg],[Could light bulbs hurt your health? One compa...,Wed Jun 10 17:34:49 +0000 2015,[Electric lights have made the world safer and...,One company will put a health notice on all th...,"health, Should there be warning labels on your...",[(CNN)The light bulb always makes the world's ...,Warning labels on your light bulbs
5,609551038983475200,[media/609551038983475201.png],[13 classic â€™00s songs that were actually me...,Sat Jun 13 02:41:07 +0000 2015,[],,,[One artistâ€™s trash is anotherâ€™s No. 1 sin...,13 Classic â€™00s Songs That Were Actually Mea...
6,609447408955719680,[media/609447408955719681.jpg],[Dez Bryant is reportedly considering skipping...,Fri Jun 12 19:49:19 +0000 2015,[],"Wide receiver Dez Bryant has signed a new,...","Football, NFL, NFC East, Dallas Cowboys, Dez B...","[Wide receiver Dez BryantÂ has signed a new, l...","Dez Bryant Contract: Latest News, Rumors, Spec..."
7,609027430624288768,[],[Pregnant mother of 12 accused of keeping kids...,Thu Jun 11 16:00:29 +0000 2015,[],A pregnant mother of 12 is due in court on chi...,,"[TULSA, Okla. â€“ Â A pregnant mother of 12 is...",Pregnant mother of 12 accused of keeping kids ...
8,608229011572068352,[],[RT @fionamatthias: 10 ways the expat life Is ...,Tue Jun 09 11:07:51 +0000 2015,[Scotland to Seek Second Independence Referend...,Thereâ€™s no autopilot when you're an expat li...,"adventure,Alienation,Bangkok,Culture,Culture S...",[Thereâ€™s no autopilot when you live abroad. ...,10 Ways the Expat Life Is Like a Continual Esp...
9,609046214554755072,[media/609046214554755073.jpg],"[House #GOP plans two days of debate, Friday s...",Thu Jun 11 17:15:07 +0000 2015,"[Obama, Chairman Paul Ryan and Republicans act...",House Republican leaders have planned for a tw...,,[House Republican leaders have planned for a t...,"House GOP plans two days of debate, Friday sho..."


In [21]:
features = TrainingSet[['id']].copy() # implemented features will be added to this matrix.

# Sentiment scores

In [23]:
#Sentiment Score VADER : of post title, target title and their differences (more info on https://github.com/cjhutto/vaderSentiment)
sid = SentimentIntensityAnalyzer()
postText = TrainingSet["postText"].tolist()
targetText = TrainingSet["targetTitle"].tolist()

#Add sentiment features to feature table
countsDF = pd.DataFrame(columns = ["sentimentPostText","sentimentTargetText","difsentimentPostTarget"])

countsDF["sentimentPostText"] = [sid.polarity_scores(sentence[0])["compound"] for sentence in postText] # vector with the sentiment score of each post text
countsDF["sentimentTargetText"] = [sid.polarity_scores(sentence)["compound"] for sentence in targetText] #vector with sentiment score of target Text
countsDF["difsentimentPostTarget"]= countsDF["sentimentPostText"] - countsDF["sentimentTargetText"]
features=pd.concat([features,countsDF],axis=1)

In [25]:
features

,id,sentimentPostText,sentimentTargetText,difsentimentPostTarget,nrTokens
0,608310377143799808,0.4019,0.4019,0.0000,15
1,609297109095972864,0.6369,-0.5423,1.1792,21
2,609504474621612032,0.3400,0.3400,0.0000,17
3,609748367049105408,0.3612,0.3612,0.0000,11
4,608688782821453824,-0.7003,-0.3400,-0.3603,18
5,609551038983475200,0.0000,0.0000,0.0000,14
6,609447408955719680,0.0000,0.0000,0.0000,19
7,609027430624288768,-0.2960,-0.2960,0.0000,12
8,608229011572068352,0.3612,0.3612,0.0000,16
9,609046214554755072,0.0000,0.0000,0.0000,16


# POS patterns

In [27]:
#Getting POS tags
#join everything
total=TrainingSet['postText'].apply(", ".join)
total = ' '.join(total.tolist())
tokenizer =  TweetTokenizer()
posTags = pd.DataFrame(nl.pos_tag(tokenizer.tokenize(total)))
posCategories = posTags[1].value_counts().index
posCategories=posCategories.tolist() # contains POS tags found in our dataset

In [29]:
#POS Bigram
best2grams = [('NNP','NNP'),('NNP','VBZ'),('IN','NNP'),('NNP',"."),('PRP','VBP'),('NN','IN'),('IN','NN'),('NNP','NNS'),('IN','JJ'),('NNP','POS'),('NN','NN'),('NN','NNP'),('NNP','VBD'),('VBN','IN'),('JJ','NNP'),('DT','NN')]
def POSBigram(sentence,first,second):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2) in nl.bigrams(posTags):
        if (t1==first and t2 == second):
            count=count+1
    return(count)

In [31]:
for pairs in best2grams:
    counts = TrainingSet['postText'].apply(POSBigram,args=(pairs[0],pairs[1]))
    colname="POS2_"+pairs[0]+"_"+pairs[1]
    countsDF = pd.DataFrame(columns = [colname])
    countsDF[colname]=counts
    features=pd.concat([features,countsDF],axis=1)

In [33]:
#POS Trigram
best3grams = [('NNP','NNP','VBZ'),('NN','IN','NNP'),('IN','NNP','NNP'),('NNP','NNP','NNP'),('NNP','NNP','NN'),('NNP','NN','NN')]
def POSTrigram(sentence,first,second,third):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2),(w3,t3) in nl.trigrams(posTags):
        if (t1==first and t2 == second and t3 == third):
            count=count+1
    return(count)

In [35]:
for tris in best3grams:
    counts = TrainingSet['postText'].apply(POSTrigram,args=(tris[0],tris[1],tris[2]))
    colname="POS3_"+tris[0]+"_"+tris[1]+"_"+tris[2]
    countsDF = pd.DataFrame(columns = [colname])
    countsDF[colname]=counts
    features=pd.concat([features,countsDF],axis=1)

In [37]:
def StartsWithNumber(sentence):
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    if posTags[0][1]=="CD":
        start=1
    else:
        start=0

    return(start)

In [39]:
countsDF = pd.DataFrame(columns = ["StartsWithNumber"])
countsDF["StartsWithNumber"]=TrainingSet['postText'].apply(StartsWithNumber)
features=pd.concat([features,countsDF],axis=1)

In [41]:
def StartsWith5WIH(sentence):
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    token_sentence=tokenizer.tokenize(sentence)
    WIH5 = ["where","what","when","which","how"]
    firstWord=token_sentence[0].lower()
    if firstWord in WIH5:
        start=1
    else:
        start=0
    return(start)

In [43]:
countsDF = pd.DataFrame(columns = ["StartsWith5WIH"])
countsDF["StartsWith5WIH"]=TrainingSet['postText'].apply(StartsWith5WIH)
features=pd.concat([features,countsDF],axis=1)

In [45]:
def containsThisNN(sentence):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2) in nl.bigrams(posTags):
        if (w1.lower()=="this" and t2 == "NN"):
            count=count+ 1
        elif (w1.lower()=="these" and t2 == "NN"):
            count=count +1 
    return(count)

In [47]:
countsDF = pd.DataFrame(columns = ["containsThisNN"])
countsDF["containsThisNN"]=TrainingSet['postText'].apply(containsThisNN)
features=pd.concat([features,countsDF],axis=1)

# Number of Tokens

In [49]:
#retrieves number of tokens in a content, using a nltk tokenizer especially suited for twitter data. punctuation is NOT removed
def numTwitterTokens(content):
    tokenizer =  TweetTokenizer()
    if content is None:
        return -1
    elif not content:
        return -1
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)

In [51]:
countsDF = pd.DataFrame(columns = ["nrTokens"])
countsDF["nrTokens"]=TrainingSet['postText'].apply(numTwitterTokens)
features=pd.concat([features,countsDF],axis=1)

In [53]:
# Other Features
def useOfSuperlative(content):
    count=0
    if type(content) is not str:
        content=content[0]
    content=content.lower()
    if bool(re.search(r'\bthe .*est( .*|$)', content)):
        count=count+1
    if bool(re.search(r'\bthe most( .*|$)', content)):
        count=count+1
    if bool(re.search(r'\bthe worst( .*|$)', content)):
        count = count+1
    if bool(re.search(r'\bthe least( .*|$)', content)):
        count=count+1
    return(count)

In [55]:
#Presence of Question Mark
def presenceQM(content):
    count=0
    if type(content) is not str:
        content=content[0]
    if "?" in content:
        count=1
    return(count)

In [57]:
#Presence of Exclamation
def presenceExc(content):
    count=0
    if type(content) is not str:
        content=content[0]
    if "!" in content:
        count=1
    return(count)

In [59]:
def numberUpperCaseWords(content):
    if type(content) is not str:
        content=content[0]
    count = len(re.findall(r'\b([A-Z]{2,})\b', content))
    return(count)

In [61]:
countsDF = pd.DataFrame(columns = ["useOfSuperlative","presenceQM","presenceExc","numberUpperCaseWords"])
countsDF["useOfSuperlative"]=TrainingSet['postText'].apply(useOfSuperlative)
countsDF["presenceQM"]=TrainingSet['postText'].apply(presenceQM)
countsDF["presenceExc"]=TrainingSet['postText'].apply(presenceExc)
countsDF["numberUpperCaseWords"]=TrainingSet['postText'].apply(numberUpperCaseWords)

features=pd.concat([features,countsDF],axis=1)

## Informality Measures (from Biyani et al.)

In [63]:
def processBodyArticle(content):
    
    content= [sentence.replace("Â\xa0", "") for sentence in content]
    content=[i for i in content if i != ""]
    content= " ".join(content)
    return(content)

In [65]:
#CL Score :0.0588 L - 0.296S -15.8 where L = avg nr of letters per 100 words, S=avg number of sentences per 100 words
#content is body of the articles
def computeInformalityScores(content):
    content = processBodyArticle(content)
    if content!="":
        sent_tokenize_list = sent_tokenize(content)
        nrSentences=len(sent_tokenize_list)
        words = re.findall(r'[a-zA-Z]+', content)
        longWords = [w for w in words if len(w)>6]
        nrLongWords = len(longWords)
        nrWords=len(words)
        nrLetters = len("".join(words))
        L = nrLetters*100/nrWords
        S = nrSentences*100/nrWords
        CLScore = 0.0588*L - 0.296*S -15.8
        RIX = nrLongWords/nrSentences
        LIX = nrWords/nrSentences + 100*nrLongWords/nrWords
        res=[CLScore,RIX,LIX]
    else:
        res = [-1,-1,-1]
    
    return res
    

In [67]:
def POSExistence(posTags,tags):
    count=0
    for (w1,t1) in posTags:
        if (t1 in tags):
            count=count+1
    return(count)

def computeFormMeasure(article):
    article = processBodyArticle(article)
    if article !="":
        #tokenization
        tokenizer =  TweetTokenizer()
        posTags = nl.pos_tag(tokenizer.tokenize(article))
        
        
        nrNouns = POSExistence(posTags,["NN","NNP","NNS"])
        nrAdj = POSExistence(posTags,["JJ","JJR","JJS"])
        nrPrep = POSExistence(posTags,["IN","TO"])
        nrArtic=POSExistence(posTags,["DT"])
        nrPron = POSExistence(posTags,["PRP","PRP$"])
        nrVerb = POSExistence(posTags,["VB","VBD","VBG","VBN","VBP","VBZ"])
        nrAdver = POSExistence(posTags,["RB","RBR","RBS","WRB"])
        nrInter = POSExistence(posTags,["UH"])

        measure = (nrNouns+nrAdj+nrPrep+nrArtic-nrPron-nrVerb-nrAdver-nrInter+100)/2
    else:
        measure=-10000000
    return measure
    

In [69]:
#join informality measures to features data frame
countsDF = pd.DataFrame(columns = ["inform_CLScore","inform_RIX","inform_LIX","inform_fmeas"])
informalityMeasures=TrainingSet['targetParagraphs'].apply(computeInformalityScores)
countsDF["inform_CLScore"]=[metric[0] for metric in informalityMeasures]
countsDF["inform_RIX"]=[metric[1] for metric in informalityMeasures]
countsDF["inform_LIX"]=[metric[2] for metric in informalityMeasures]
countsDF["inform_fmeas"]=TrainingSet['targetParagraphs'].apply(computeFormMeasure)
features=pd.concat([features,countsDF],axis=1)

## Unigrams and Bigrams from PostTitles

In [71]:
from collections import Counter
from nltk.util import ngrams
from nltk.stem import *
from sklearn.feature_extraction.text import CountVectorizer # count unigrams

In [73]:
#Retrieve words from postTitles
postTitles = TrainingSet["postText"].tolist()

def stemmingPostTitles(postTitles,stem):
    tokenizer =  TweetTokenizer()
    if (stem=="porter"):
        stemmer = nl.PorterStemmer() # Porter Stemming
    elif (stem == "snowball"):
        stemmer = SnowballStemmer("english")
    elif (stem == "lancaster"):
        stemmer = LancasterStemmer()
    
    number_pattern = re.compile(r'\d+')
    corpus=[]
    for title in postTitles:
        title = title[0]
        
        title=title.lower()
        tokens = tokenizer.tokenize(title)
        stem_tokens = [stemmer.stem(t) for t in tokens ]
        stem_title = " ".join(stem_tokens)
        stem_title = number_pattern.sub('[N]', stem_title)
        corpus.append(stem_title)
    return(corpus)

def getUniAndBigramsFeatures(stem):
    corpusPostTitles=stemmingPostTitles(postTitles,stem)
    cv_kw = CountVectorizer(token_pattern='\\S+', ngram_range=(1, 2), min_df=10,  # at least 10 occurences
                     binary=False, dtype=np.uint8)
    X_kw = cv_kw.fit_transform(corpusPostTitles)
    df=pd.DataFrame(X_kw.toarray())
    featurenames = [re.sub(" ","_",name) for name in cv_kw.get_feature_names()]
    df.columns = [stem+"_"+feature for feature in featurenames]
    return(df)
    

In [75]:
uniBig=getUniAndBigramsFeatures("porter") # atm we are only using porter stemmer, we can see if changing the stemmer has any impact on the result.

In [77]:
features=pd.concat([features,uniBig],axis=1)

In [ ]:
features